## Preparing Population Growth Rate Before and Since Pandemic Dataset
#### By Xinqian Zhai, Cliff Gong, Gen Ho
- 2018 population at county level. Data from: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html 
- 2021 population at county level. Data from: https://worldpopulationreview.com/us-counties/states/ca 

In [1]:
# import useful libraries
import pandas as pd 
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
# get 2018 population at county level for the three states
def get_population18_data():
    state_prefix = [('CA',' California'), 
                    ('TX',' Texas'), 
                    ('NY',' New York')]
    df_list = []
    for item in state_prefix:
        df =  pd.read_csv('./demographic_data/population'+item[0]+'18.csv', header = 3, 
                            usecols = ['Unnamed: 0','2018'], skipfooter = 5)
        df = df.iloc[1:, : ]
        df.columns = ['county','2018']
        df['county'] = df['county'].str[1:].replace(item[1], item[0], regex = True)
        df_list.append(df) 
    df = pd.concat(df_list).reset_index(drop=True)
    df['2018'] = df['2018'].str.split(',').str.join('').astype(int)

    return df

In [3]:
# web scrap 2021 population data at county level for three states
def get_population21_table(tax_state):
    state_prefix = {'CA':'ca',
                    'NY':'ny',
                    'TX':'tx'}
    url_link = 'https://worldpopulationreview.com/us-counties/states/{}'.format(state_prefix[tax_state])

    r = requests.get(url_link)
    html_to_dataframe = pd.read_html(r.text)[0]

    # only get the 2021 population
    df = html_to_dataframe.iloc[:, [0,1]]
    df.columns = ['county', '2021']
    df['county'] = df['county'] + ',' + tax_state
    df = df.sort_values('county', ascending=True).reset_index(drop=True)

    return df

# get population for the states
pop21_ca = get_population21_table('CA')
pop21_ny = get_population21_table('NY')
pop21_tx = get_population21_table('TX')

# combine all to form one dtaframe
pop21_all = pop21_ca.append(pop21_ny, ignore_index=True).append(pop21_tx, ignore_index=True)

In [4]:
# combine population 2018 and 2021 to calculate population growth rate
def get_pop_growth_rate():
    pop18_all = get_population18_data()

    population = pop18_all.merge(pop21_all, on = 'county')
    population['growth_rate'] = round((population['2021'] - population['2018'])/population['2018'], 3)
    
    # align populations of counties with the counties in the rentals, which only keep 49 counties
    returns = pd.read_csv("./output/returns.csv", index_col=0)
    counties_to_keep = returns.columns.values
    population = population[population['county'].isin(counties_to_keep)]

    return population

pop_growth_rate = get_pop_growth_rate()

In [5]:
# write population growth rate to demographic_data folder
pop_growth_rate.to_csv('./demographic_data/pop_growth_rate.csv', index = False)

In [6]:
df = pd.read_csv('./demographic_data/pop_growth_rate.csv')
df.head()


,county,2018,2021,growth_rate
0,"Alameda County,CA",1666756,1680480,0.008
1,"Contra Costa County,CA",1150519,1159540,0.008
2,"Kern County,CA",893758,913090,0.022
3,"Los Angeles County,CA",10073906,9969510,-0.010
4,"Marin County,CA",259662,257154,-0.010


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=718fdf0e-933b-4ec5-90a8-5b2fe887b720' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>